In [0]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/sentiment')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Data Preprocessing: tokenize, lowercase, padding

Utilize torchtext to load train and test data. Define batch of examples.



In [0]:
import load_review
import torch
import torch.nn.functional as F
import torch.optim as optim
from torchtext.data import TabularDataset
from torchtext import data
from torchtext.vocab import GloVe
tokenize = lambda x: x.split()
TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True, include_lengths=True, batch_first=True,
                      fix_length=200)
LABEL = data.LabelField()
train_data, test_data = TabularDataset.splits(path='', train='train.csv', test='test.csv', format='csv',
                                                  fields=[('text', TEXT), ('label', LABEL)])
TEXT.build_vocab(train_data, vectors=GloVe(name='6B', dim=300))
LABEL.build_vocab(train_data)
word_embeddings = TEXT.vocab.vectors
print("Length of Text Vocabulary: " + str(len(TEXT.vocab)))
print("Vector size of Text Vocabulary: ", TEXT.vocab.vectors.size())
print("Label Length: " + str(len(LABEL.vocab)))

train_iter, test_iter = data.BucketIterator.splits((train_data, test_data), batch_size=32,
                                                        sort_key=lambda x: len(x.text), repeat=False,
                                                        shuffle=True)
vocab_size = len(TEXT.vocab)

Model definition. Embedding layer: 

*   Embedding layer: self.lookup_table, initialize with 300 dimensional Glove word embeddings
*   BiLSTM layer:  self.lstm
*   Attention
*   Max pooling
*   Average pooling




In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F
import numpy as np


class bilstm_attn(torch.nn.Module):
    def __init__(self, batch_size, output_size, hidden_size, vocab_size, embed_dim, weights, bidirectional, dropout,
                 use_cuda, attention_size, sequence_length):
        super(bilstm_attn, self).__init__()

        self.batch_size = batch_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.bidirectional = bidirectional
        # self.dropout = dropout
        self.use_cuda = use_cuda
        self.sequence_length = sequence_length
        self.lookup_table = nn.Embedding(self.vocab_size, self.embed_dim)
        # self.lookup_table.weight.data.uniform_(-1., 1.)
        self.lookup_table.weight = nn.Parameter(weights, requires_grad=False)

        self.layer_size = 1
        self.lstm = nn.LSTM(self.embed_dim,
                            self.hidden_size,
                            self.layer_size,
                            bidirectional=self.bidirectional)

        if self.bidirectional:
            self.layer_size = self.layer_size * 2
        else:
            self.layer_size = self.layer_size

        self.attention_size = attention_size
        if self.use_cuda:
            self.w_omega = Variable(torch.zeros(self.hidden_size * self.layer_size, self.attention_size).cuda())
            self.u_omega = Variable(torch.zeros(self.attention_size).cuda())
        else:
            self.w_omega = Variable(torch.zeros(self.hidden_size * self.layer_size, self.attention_size))
            self.u_omega = Variable(torch.zeros(self.attention_size))

        self.label = nn.Linear(hidden_size * self.layer_size *3, output_size)
        self.dropout = nn.Dropout(dropout)
        self.max_pool = nn.AdaptiveMaxPool1d(1)
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.softmax = nn.Softmax(dim=1)
        self.init_weights()
        # init_lstm( self.lstm )
   

    # self.attn_fc_layer = nn.Linear()

    def init_weights(self):
      initrange = 0.5
      self.label.weight.data.uniform_(-initrange, initrange)
      self.label.bias.data.zero_()

    def attention_net(self, lstm_output):
        # print(lstm_output.size()) = (squence_length, batch_size, hidden_size*layer_size)

        output_reshape = torch.Tensor.reshape(lstm_output, [-1, self.hidden_size * self.layer_size])
        # print(output_reshape.size()) = (squence_length * batch_size, hidden_size*layer_size)

        attn_tanh = torch.tanh(torch.mm(output_reshape, self.w_omega))
        # print(attn_tanh.size()) = (squence_length * batch_size, attention_size)

        attn_hidden_layer = torch.mm(attn_tanh, torch.Tensor.reshape(self.u_omega, [-1, 1]))
        # print(attn_hidden_layer.size()) = (squence_length * batch_size, 1)

        exps = torch.Tensor.reshape(torch.exp(attn_hidden_layer), [-1, self.sequence_length])
        # print(exps.size()) = (batch_size, squence_length)

        alphas = exps / torch.Tensor.reshape(torch.sum(exps, 1), [-1, 1])
        # print(alphas.size()) = (batch_size, squence_length)

        alphas_reshape = torch.Tensor.reshape(alphas, [-1, self.sequence_length, 1])
        # print(alphas_reshape.size()) = (batch_size, squence_length, 1)

        state = lstm_output.permute(1, 0, 2)
        # print(state.size()) = (batch_size, squence_length, hidden_size*layer_size)

        attn_output = torch.sum(state * alphas_reshape, 1)
        # print(attn_output.size()) = (batch_size, hidden_size*layer_size)

        return attn_output

    def forward(self, input_sentences, batch_size=None):
        input = self.lookup_table(input_sentences)
        input = self.dropout(input)
        input = input.permute(1, 0, 2)
        if batch_size is None:
          current_batch_size = self.batch_size
        else:
          current_batch_size = batch_size

        if self.use_cuda:
            h_0 = Variable(torch.zeros(self.layer_size, current_batch_size, self.hidden_size).cuda())
            c_0 = Variable(torch.zeros(self.layer_size, current_batch_size, self.hidden_size).cuda())
        else:
            h_0 = Variable(torch.zeros(self.layer_size, current_batch_size, self.hidden_size))
            c_0 = Variable(torch.zeros(self.layer_size, current_batch_size, self.hidden_size))

        lstm_output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0))
        lstm_output = self.dropout(lstm_output)
        max_p = self.max_pool(lstm_output.permute(1,2,0)).squeeze(2)
        avg_p = self.avg_pool(lstm_output.permute(1,2,0)).squeeze(2)
        attn_output = self.attention_net(lstm_output)
        x= torch.cat((attn_output,max_p,avg_p),dim=1)
        logits = self.label(x)
        # output = self.softmax(logits)
        return logits,attn_output


Training and evaluation part.

In [0]:
import time
from tqdm import tqdm

train_loss = []
valid_loss = []
accuracy = []

def train_model(model, train_iter, epoch):
  total_epoch_loss = 0
  total_epoch_acc = 0
  steps = 0
  model.train()
  for idx, batch in enumerate(train_iter):
    text = batch.text[0]
    target = batch.label
    target = torch.autograd.Variable(target).long()
    if use_cuda:
      text = text.cuda()
      target = target.cuda()
      if (text.size()[0] is not 32):
        # One of the batch returned by BucketIterator has length different than 32.
        continue
      optimizer.zero_grad()
      prediction,_ = model(text)
      loss = criterion(prediction, target)
      num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).float().sum()
      acc = 100.0 * num_corrects/len(batch)
      loss.backward()
      optimizer.step()
      steps += 1
      if steps % 100 == 0:
            print (f'Epoch: {epoch+1}, Idx: {idx+1}, Training Loss: {loss.item():.4f}, Training Accuracy: {acc.item(): .2f}%')
        
      total_epoch_loss += loss.item()
      total_epoch_acc += acc.item()
  return total_epoch_loss/len(train_iter), total_epoch_acc/len(train_iter)

def eval_model(model,val_iter):
  total_epoch_loss = 0
  total_epoch_acc = 0
  model.eval()
  with torch.no_grad():
      for idx, batch in enumerate(val_iter):
          text = batch.text[0]
          if (text.size()[0] is not 32):
              continue
          target = batch.label
          target = torch.autograd.Variable(target).long()
          if torch.cuda.is_available():
              text = text.cuda()
              target = target.cuda()
          prediction,_ = model(text)
          loss = criterion(prediction, target)
          num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).sum()
          acc = 100.0 * num_corrects/len(batch)
          total_epoch_loss += loss.item()
          total_epoch_acc += acc.item()
  return total_epoch_loss/len(val_iter), total_epoch_acc/len(val_iter)

Initialize bilstm_attn model with pre-defined parameters. Define the opimizer Adam. Set loss function without weights.

In [0]:
lr=1e-3
batch_size = 32
output_size = 3
hidden_size = 128
embedding_length = 300
num_layers=1
seed=12
dropout=0.5
torch.manual_seed(seed)
use_cuda = torch.cuda.is_available()
sequence_length=200 
attention_size=200
model = bilstm_attn(batch_size, output_size, hidden_size, vocab_size, embedding_length, word_embeddings, True, dropout,
                    use_cuda, sequence_length, attention_size)
if use_cuda:
    model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
class_weights = torch.FloatTensor().cuda()
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

Training log of 'best_model_hidden_128_eval.pt'.

In report, the balanced model is *Epoch: 15, Train Loss: 0.356, Train Acc: 86.10%, Val. Loss: 0.379567, Val. Acc: 85.40%*.


In [0]:
epoches = 100
best_val_loss=None
for epoch in range(epoches):
  train_loss, train_acc = train_model(model, train_iter, epoch)
  val_loss, val_acc = eval_model(model, test_iter)
  print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%, Val. Loss: {val_loss:3f}, Val. Acc: {val_acc:.2f}%')
  if not best_val_loss or val_loss<best_val_loss:
    with open('best_model_hidden_128_eval.pt','wb') as f:
      torch.save(model.state_dict(),f)
      print('save model successfully at epoch: '+ str(epoch+1))
    best_val_loss = val_loss

Epoch: 1, Idx: 100, Training Loss: 0.8046, Training Accuracy:  78.12%
Epoch: 1, Idx: 200, Training Loss: 0.5711, Training Accuracy:  78.12%
Epoch: 1, Idx: 300, Training Loss: 0.5745, Training Accuracy:  78.12%
Epoch: 1, Idx: 400, Training Loss: 0.6609, Training Accuracy:  78.12%
Epoch: 1, Idx: 500, Training Loss: 0.5957, Training Accuracy:  78.12%
Epoch: 1, Idx: 600, Training Loss: 0.8350, Training Accuracy:  65.62%
Epoch: 1, Idx: 700, Training Loss: 0.5793, Training Accuracy:  78.12%
Epoch: 1, Idx: 800, Training Loss: 0.5419, Training Accuracy:  84.38%
Epoch: 1, Idx: 900, Training Loss: 0.6966, Training Accuracy:  75.00%
Epoch: 1, Idx: 1000, Training Loss: 0.4667, Training Accuracy:  87.50%
Epoch: 1, Idx: 1100, Training Loss: 0.4537, Training Accuracy:  81.25%
Epoch: 1, Idx: 1200, Training Loss: 0.6384, Training Accuracy:  71.88%
Epoch: 1, Idx: 1300, Training Loss: 0.5239, Training Accuracy:  84.38%
Epoch: 1, Idx: 1400, Training Loss: 0.6329, Training Accuracy:  75.00%
Epoch: 1, Idx: 

KeyboardInterrupt: ignored

Weighted loss model setting.
Add weights list [positive,negative,neutral] =[1.0,6.07,6.88] to the Adam cross entropy loss funtion

In [0]:
if use_cuda:
    model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
weights = [1.0,6.07,6.88]
class_weights = torch.FloatTensor(weights).cuda()
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

Training log of 'best_model_hidden_128_eval_weighted.pt'. 

If you follow this file and want to train the model, you need to initialise the model and run .cuda() again.

In report, the weighted model : 
*Epoch: 30, Train Loss: 0.549, Train Acc: 79.99%, Val. Loss: 0.634149, Val. Acc: 79.96%*

In [0]:
epoches = 100
best_val_loss=None
for epoch in range(epoches):
  train_loss, train_acc = train_model(model, train_iter, epoch)
  val_loss, val_acc = eval_model(model, test_iter)
  print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%, Val. Loss: {val_loss:3f}, Val. Acc: {val_acc:.2f}%')
  if not best_val_loss or val_loss<best_val_loss:
    with open('best_model_hidden_128_eval_weighted.pt','wb') as f:
      torch.save(model.state_dict(),f)
      print('save model successfully at epoch: '+ str(epoch+1))
    best_val_loss = val_loss

Epoch: 1, Idx: 100, Training Loss: 1.0297, Training Accuracy:  53.12%
Epoch: 1, Idx: 200, Training Loss: 1.2671, Training Accuracy:  43.75%
Epoch: 1, Idx: 300, Training Loss: 1.0126, Training Accuracy:  43.75%
Epoch: 1, Idx: 400, Training Loss: 0.9103, Training Accuracy:  46.88%
Epoch: 1, Idx: 500, Training Loss: 0.9630, Training Accuracy:  75.00%
Epoch: 1, Idx: 600, Training Loss: 0.6853, Training Accuracy:  59.38%
Epoch: 1, Idx: 700, Training Loss: 0.8773, Training Accuracy:  68.75%
Epoch: 1, Idx: 800, Training Loss: 0.7251, Training Accuracy:  56.25%
Epoch: 1, Idx: 900, Training Loss: 1.3656, Training Accuracy:  37.50%
Epoch: 1, Idx: 1000, Training Loss: 1.0910, Training Accuracy:  68.75%
Epoch: 1, Idx: 1100, Training Loss: 1.2454, Training Accuracy:  59.38%
Epoch: 1, Idx: 1200, Training Loss: 0.6922, Training Accuracy:  71.88%
Epoch: 1, Idx: 1300, Training Loss: 0.9207, Training Accuracy:  62.50%
Epoch: 1, Idx: 1400, Training Loss: 0.7749, Training Accuracy:  56.25%
Epoch: 1, Idx: 